In [53]:
import os
import openai

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read the .env file in local directory
openai.api_key = os.environ[
    "OPENAI_API_KEY"
]  # to use this notebook we must set up an API key from OpenAI

llm_model = "gpt-3.5-turbo-0125" 

In [2]:
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create( # we are requesting a completion from the chat endpoint
        model=model, # here we are specifying the model we are going to use
        messages=messages, # here we are passing the messages to the model
        temperature=0.0, # here we are setting the temperature to 0.0, temperature is a hyperparameter that controls the randomness of the output. if we set temperature to higher numbers like 0.7 or 1.0 the output will be more random in other words probability of the correct answer will be lower.
        n=3, # here we are setting the number of completions we want to get from the model let's check how responses will differ
    )
    if not response.choices:
        raise ValueError("No response choices returned from the API.")
    return (response.choices[0].message.content, 
            response.choices[1].message.content, 
            response.choices[2].message.content) # here we are returning the content of the response


In [3]:
response = get_completion("What is the capital of France?")
print(response)  # expected output: 'Paris'

('The capital of France is Paris.', 'The capital of France is Paris.', 'The capital of France is Paris.')


In [4]:
# !pip install -U langchain-openai 
# to use openAI via langchain we need to install langchain-openai

In [5]:
from langchain_openai import ChatOpenAI

In [6]:
chat = ChatOpenAI(temperature=0.0, n=3) # creating chat instance using openai
chat

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000011651BD8890>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001165112C250>, root_client=<openai.OpenAI object at 0x0000011651728C90>, root_async_client=<openai.AsyncOpenAI object at 0x0000011651A98110>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), n=3)

In [7]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

# here we are instantiated our template_string which will be use in later sections. 
# if you look closely {style} and {text} arguments in curly braces, we'll see how to use it later  

In [8]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

# here we created chatprompttemplate this is a template for communicate between user and model

In [9]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [10]:
prompt_template.messages[0].prompt.input_variables # here prompt_tempalte received curly braces as input variables

['style', 'text']

In [11]:
customer_style = """American English \
in a calm and respectful tone
"""

In [12]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [13]:
customer_messages = prompt_template.format_messages(
    style=customer_style,
    text=customer_email
)

# we specified which style and text we will pass to prompt template 

In [14]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [15]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} response_metadata={}


In [16]:
customer_response = chat(customer_messages)

C:\Users\AHMET\AppData\Local\Temp\ipykernel_29464\3256373024.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  customer_response = chat(customer_messages)


In [17]:
print(customer_response.content) # as we can see customer message translated into daily english

Oh man, I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And on top of that, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, buddy.


In [18]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [19]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [20]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

# here we are doing same thing but reverse we answer customer in their tone (language)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [21]:
service_response = chat(service_messages)
print(service_reply)
print("----------------------")
print(service_response.content)

Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!

----------------------
Ahoy there, valued customer! Regrettably, the warranty be not coverin' the cost o' cleanin' yer galley due to yer own negligence. Ye see, 'twas yer own doin' that ye forgot to secure the lid afore startin' the blender. 'Tis a tough break, indeed! Fare thee well!


In [22]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

# here we declared a review template and we specified how the LLM should return it should return similar to this JSON 

# {
#   "gift": False,
#   "delivery_days": 5,
#   "price_value": "pretty affordable!"
# }

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [23]:
from langchain.prompts import ChatPromptTemplate

# here we are again create chatprompttemplate by parsing review template.
# we didn't only need to specify text variable not others because the other ones will be determined by LLM
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [54]:
messages = prompt_template.format_messages(text=customer_review) # we give customer_review to LLm
chat = ChatOpenAI(temperature=0.0, model=llm_model)
response = chat(messages)
print(response.content)

```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": [
    "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
  ]
}
```


In [25]:
type(response.content)

str

In [26]:
# be careful despite LLM returns answer in JSON "format" not actual JSON object
# so when we try something like this we get an error because LLM knows what JSON format is but it doesn't return in JSON format

response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

In [27]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [61]:
# but we can overcome this issue by just using response schema 
# now we defined what parser should return according to the description they are given

gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased \n \
                             as a gift for someone else? \n \
                             Answer True if yes, \n \
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\n \
                                      did it take for the product\n \
                                      to arrive? If this \n \
                                      information is not found,\n \
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\n \
                                    sentences about the value or \n \
                                    price, and output them as a \n \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [62]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas) 
# here we declared how structured outputs should be it will take into account response schema elements

In [63]:
format_instructions = output_parser.get_format_instructions() 
# we formatted them same as format_messages the only difference is now we format instructions

In [64]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased 
                              as a gift for someone else? 
                              Answer True if yes, 
                              False if not or unknown.
	"delivery_days": string  // How many days
                                       did it take for the product
                                       to arrive? If this 
                                       information is not found,
                                       output -1.
	"price_value": string  // Extract any
                                     sentences about the value or 
                                     price, and output them as a 
                                     comma separated Python list.
}
```


In [65]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [66]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [67]:
response = chat(messages)

In [68]:
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [69]:
output_dict = output_parser.parse(response.content)

In [70]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [71]:
type(output_dict)

dict

In [72]:
output_dict.get('delivery_days')

'2'